In [13]:
import pandas as pd
df = pd.read_csv('scoreData.csv')
step = df.loc[df.notna()['weather_detail'] == True]
step.loc[step['weather_detail'] != "DOME"]


,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
9774,9/27/2009,2009,3,False,Baltimore Ravens,34.0,3.0,Cleveland Browns,BAL,-13.5,38.5,M&T Bank Stadium,False,71.0,8.0,87,Rain
9780,9/27/2009,2009,3,False,New England Patriots,26.0,10.0,Atlanta Falcons,NE,-4.5,45.5,Gillette Stadium,False,64.0,14.0,84,Rain
9781,9/27/2009,2009,3,False,New York Jets,24.0,17.0,Tennessee Titans,NYJ,-1.0,36,Giants Stadium,False,66.0,8.0,85,Rain | Fog
9783,9/27/2009,2009,3,False,Philadelphia Eagles,34.0,14.0,Kansas City Chiefs,PHI,-7.5,39,Lincoln Financial Field,False,68.0,9.0,88,Fog
9797,10/4/2009,2009,4,False,New England Patriots,27.0,21.0,Baltimore Ravens,NE,-2.0,45,Gillette Stadium,False,65.0,5.0,89,Fog
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12354,12/15/2018,2018,15,False,New York Jets,22.0,29.0,Houston Texans,HOU,-7.0,44,MetLife Stadium,False,51.0,8.0,NaN,Rain
12356,12/16/2018,2018,15,False,Baltimore Ravens,20.0,12.0,Tampa Bay Buccaneers,BAL,-8.5,45,M&T Bank Stadium,False,43.0,10.0,NaN,Rain
12357,12/16/2018,2018,15,False,Buffalo Bills,14.0,13.0,Detroit Lions,BUF,-2.5,40,New Era Field,False,39.0,7.0,NaN,Rain
12364,12/16/2018,2018,15,False,New York Giants,0.0,17.0,Tennessee Titans,TEN,-2.5,42,MetLife Stadium,False,40.0,6.0,NaN,Rain
